### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [31]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
model_client = OllamaChatCompletionClient(
    model="qwen3",
)

In [32]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(
    model="llama3.3:latest",
)

### Second concept: The Message

In [33]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 16, 2, 5, 8, 655317, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [34]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [35]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'<think>\nOkay, the user wants to go to London. I need to respond in a short, humorous way. Let me think about common airline jokes or puns related to London.\n\nMaybe something about the weather? Like, "Pack an umbrella... and a sense of humor, because London\'s weather is as unpredictable as my flight delays." That\'s a bit long. Let me shorten it. \n\nOr maybe reference the Tube? "Don\'t worry about the weather, just bring a map—London\'s Tube system is a maze even for locals!" Hmm, that\'s good. \n\nWait, the user might be looking for a more lighthearted comment. Maybe combine both ideas. "Pack an umbrella and a sense of humor—London\'s weather is as unpredictable as my flight delays, but the tea is always on point!" That\'s better. It\'s short, funny, and touches on two common London experiences. Let me check if it\'s within the character limit and flows well. Yeah, that works.\n</think>\n\n"Pack an umbrella and a sense of humor—London\'s weather is as unpredictable as my flight d

### Let's make a local database of ticket prices

In [36]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [37]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [38]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [39]:
get_city_price("Rome")

499.0

In [40]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [41]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

<think>
Okay, the user wants to go to London. I need to find the roundtrip ticket price. Let me check the available functions. There's get_city_price which requires the city_name. So I should call that function with "London" as the parameter. Let me make sure the JSON is correctly formatted. The function name is get_city_price and the argument is city_name: London. Alright, that should do it.
</think>


[FunctionCall(id='0', arguments='{"city_name": "London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='0', is_error=False)]


'<think>\nOkay, the user wants to go to London. I need to respond with a short, humorous answer and include the price of a round-trip ticket. The previous interaction shows that when the user says "I\'d like to go to London," the system calls a function to get the price, which returned 299.0.\n\nFirst, I should acknowledge their desire to travel. Then add a funny twist. Maybe something about the weather or the famous landmarks. The price is $299, so I should format that clearly. Let me think of a joke related to London. Maybe mention Big Ben or the weather. Oh, London\'s weather is often unpredictable, so a joke about that could work. \n\nPutting it all together: Start with a cheerful greeting, mention the price, add a humorous comment about the weather or a landmark, and end with a friendly note. Make sure it\'s concise and matches the tone of the example response. Check for any typos and ensure the price is correctly formatted. Alright, that should do it.\n</think>\n\n"Ah, London! Th